<a href="https://colab.research.google.com/github/CristhianHenry/Chatbot_medico/blob/main/Progectochatbotmedico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install tflearn

     |████████████████████████████████| 112kB 4.9MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127300 sha256=276baf03580cb347e2a74593bdaf3a7adf790c2ddff09ff69a246bc978ce1eba
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [4]:
import nltk #para trabajar con procesamiento de lenguaje natural
from nltk.stem.lancaster import LancasterStemmer #transformar palabras
stemmer = LancasterStemmer() #objeto
import numpy
import tflearn
import tensorflow
import json
import random #escoger una respuesta aleatoria

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
with open("cont.json", encoding='utf-8') as archivo:#abrimos contenido
  datos = json.load(archivo)

In [8]:
palabras=[] #guarda toda las palabras 
tags=[]          #guarda los tags 
auxX=[]      #guarda las preguntas
auxY=[]    #guarda los tags para cada pregunta

In [9]:
for contenido in datos["contenido"]:     #entra en el contenidos del json
  for patrones in contenido["patrones"]: #busca los patrones o preguntas del json
    auxPalabra = nltk.word_tokenize(patrones)#tokeniza los patrones y las guarda temporalmente en auxPalabra
    palabras.extend(auxPalabra)     #auxPalabra tokenoizada se guarda en palabras
    auxX.append(auxPalabra)          #para guardar las palabras como frases
    auxY.append(contenido["tag"])  #para guardar tags repetidos

    if contenido["tag"] not in tags:     #en la lista tags se guardan por una sola vez los tags a diferencia de en auxY
      tags.append(contenido["tag"])

print(palabras)     # muestra las palabras no procesadas
print(auxX)          # muestra las frases de preguntas en tokens
print(auxY)        # muestra los tags repetidos por cada frase
print(tags)         #muestra los tags

['Hola', 'te', 'saludo', 'que', 'tal', 'hello', 'ola', 'adios', 'hasta', 'luego', 'nos', 'vemos', 'cuidate', 'bye', 'como', 'te', 'llamas', 'cual', 'es', 'tu', 'nombre', 'con', 'quien', 'hablo', 'como', 'es', 'tu', 'nombre', 'quien', 'eres', 'como', 'estas', 'que', 'haces', 'a', 'que', 'te', 'dedicas', 'que', 'estas', 'haciendo', 'que', 'eres', 'que', 'sabes', 'hacer', 'me', 'siento', 'enfermo', 'estoy', 'mal', 'me', 'duele', 'la', 'cabeza', 'siento', 'dolor', 'de', 'cabeza', 'tengo', 'una', 'jaqueca', 'cabesa', 'tengo', 'un', 'dolor', 'de', 'cabeza', 'migrañoso', 'migrañoso', 'migraña', 'tengo', 'un', 'dolor', 'de', 'cabeza', 'en', 'racimo', 'racimo', 'por', 'racimo', 'rasimo', 'tengo', 'dolor', 'de', 'cabeza', 'por', 'tensión', 'tension', 'por', 'tension', 'tencion', 'sabes', 'multiplicar', 'dividir', 'sumar', 'o', 'restar', 'sabes', 'hacer', 'operaciones', 'matematicas', 'sabes', 'que', 'hora', 'es', 'como', 'esta', 'el', 'clima', 'tienes', 'vida', 'tienes', 'sentimientos', 'cuantos

In [11]:
palabras = [stemmer.stem(w.lower()) for w in palabras if w!="?"] #para pasar las palabras a minusculas 
palabras = sorted(list(set(palabras)))#para ordenas las palabras
tags = sorted(tags) #ordeno los tags  
#listas la procesadas
print(palabras)
print(tags)         
print(auxX)    

['a', 'adio', 'año', 'bot', 'bye', 'cabes', 'cabez', 'chatbot', 'clim', 'como', 'con', 'cread', 'cual', 'cuanto', 'cuid', 'de', 'dedica', 'diseño', 'dividir', 'dol', 'duel', 'el', 'en', 'enfermo', 'er', 'es', 'est', 'esta', 'estoy', 'hablo', 'hac', 'haciendo', 'hast', 'hello', 'hol', 'hor', 'jaquec', 'la', 'llama', 'luego', 'mal', 'matematica', 'me', 'migrañ', 'migrañoso', 'multiplic', 'nombr', 'nos', 'o', 'ol', 'operac', 'por', 'que', 'qui', 'racimo', 'rasimo', 'rest', 'robot', 'sab', 'saludo', 'sentimiento', 'siento', 'sum', 'tal', 'te', 'tend', 'tengo', 'tensión', 'tent', 'tien', 'tu', 'un', 'vemo', 'vid']
['despedida', 'estado', 'nombre', 'rIIIecabeza', 'rIIecabeza', 'rIecabeza', 'rafirmativas', 'rcabeza', 'restado', 'rnegativas', 'saludo']
[['Hola'], ['te', 'saludo'], ['que', 'tal'], ['hello'], ['ola'], ['adios'], ['hasta', 'luego'], ['nos', 'vemos'], ['cuidate'], ['bye'], ['como', 'te', 'llamas'], ['cual', 'es', 'tu', 'nombre'], ['con', 'quien', 'hablo'], ['como', 'es', 'tu', 'no

In [12]:
entrenamiento=[] #lista vacia que se utilizara para la cubeta de 0 y 1
salida=[]        #
salidaVacia=[0 for _ in range(len(tags))]#representa el tamaño de los tags
for x, documento in enumerate(auxX): #se busca las frases de preguntas tokenizadas y se guarda las frases en el documento donde se asignaran posiciones como por ejemplo el documento = hola y la x=0 todo en la lista
  cubeta=[]#alli es donde se volcará la cubeta
  auxPalabra=[stemmer.stem(w.lower()) for w in documento]#las frases patron se ponen en minuscula  y guardando en auxPalabra
  for w in palabras: #se busca cohincidencia entre las palabras para poder ubicarlo en la lista  
    if w in auxPalabra:   #si encuentra entonces agrega al cubeta asi se van creando una lista 
      cubeta.append(1)#
    else:#
      cubeta.append(0)#    #si no encuentra no agrega nada a la lista
  filaSalida = salidaVacia[:] #para asignar salidavacia a la filasalida
  filaSalida[tags.index(auxY[x])] = 1 #coloca 1 a la lista de tag que le encontró

  entrenamiento.append(cubeta) #agrega las listas cubeta
  salida.append(filaSalida)   #agrefa las listas 
print(auxPalabra) 
print(cubeta)            
print(filaSalida)             
print(entrenamiento)    
print(salida)           


['er', 'un', 'bot']
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [13]:
entrenamiento = numpy.array(entrenamiento)#pasa la lista entrenamiento a un array
salida = numpy.array(salida)              #pasa la lsita salida a un array
tensorflow.compat.v1.reset_default_graph()#limpia el graph por defecto
#modelo red neuronal
red = tflearn.input_data(shape=[None,len(entrenamiento[0])])#red con ninguna forma 
red = tflearn.fully_connected(red,73)#capas con neuronas
red = tflearn.fully_connected(red,73)
red = tflearn.fully_connected(red,len(salida[0]),activation="softmax")#la salida de tags es con funcion de activacion softmax

red = tflearn.regression(red)# regresion para obtener probaliidaides 
modelo = tflearn.DNN(red)# para entrenar el modelo con DNN
modelo.fit(entrenamiento, salida, n_epoch=250, batch_size=192, show_metric=True)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
---------------------------------
Run id: J75TR8
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 50
Validation samples: 0
--
Training Step: 1  | time: 0.406s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 50/50
--
Training Step: 2  | total loss: 2.15802 | time: 0.008s
| Adam | epoch: 002 | loss: 2.15802 - acc: 0.0540 -- iter: 50/50
--
Training Step: 3  | total loss: 2.35318 | time: 0.003s
| Adam | epoch: 003 | loss: 2.35318 - acc: 0.2225 -- iter: 50/50
--
Training Step: 4  | total loss: 2.38475 | time: 0.007s
| Adam | epoch: 004 | loss: 2.38475 - acc: 0.2956 -- iter: 50/50
--
Training Step: 5  | total loss: 2.39301 | time: 0.007s
| Adam | epoch: 005 | loss: 2.39301 - acc: 0.2017 -- iter: 50/50
--
Training Step: 6  | total loss: 2.3928

In [17]:

def mainBot():
  while True:
    entrada = input("humano: ")
    if entrada != "adios": #para salir 
      cubeta = [0 for _ in range(len(palabras))]#lleno el cubeta de 0 para reinicializar
      entradaProcesada = nltk.word_tokenize(entrada)# se tokenizo la entrada
      entradaProcesada = [stemmer.stem(palabra.lower()) for palabra in entradaProcesada]#stemizo la entrada
      for palabraIndividual in entradaProcesada:    #buscar con palabra individual la lista palabras
        for i,palabra in enumerate(palabras):  
          if palabra == palabraIndividual:          #si encuentro cohincidencia entonces agrego al cubeta las posiciones
            cubeta[i] = 1 #
      resultados = modelo.predict([numpy.array(cubeta)])#Tomamos el modelo red como parámetro 
      if numpy.amax(resultados)>0.5: #para poder encontrar la palabra tiene que ser mayor 0.5
        resultadoIndices = numpy.argmax(resultados)#  argmax retorna el INDICE del mayor elelmto

        tag = tags[resultadoIndices]#asigna tag al cual tiene mas posiblidades 
        for tagAux in datos["contenido"]: #busca el contenido del json
          if tagAux["tag"] == tag:        #en el json cuando encuentre al tag entonces asignara respuestas 
            respuesta = tagAux["respuestas"]#

        print("Bot: ",random.choice(respuesta))#elige aleatoriamente cualquera de esas posibles resuestas  

      else:#si no hay certeza del mayor a 50% entonces responde 
        print("Bot: ",random.choice(["no estoy muy seguro de la respuesta","no tengo certerza de ello"]))

    else:
      print("Bot: ",random.choice(["hasta luego","fue un gusto"]))
      break

mainBot()

humano: ola
Bot:  Hola que tal!
humano: me siento mal
Bot:  está bien puedes decirme que esta mal ¿cuáles son tus síntomas?
humano: me duele la cabesa
Bot:  qué tipo de dolor de cabeza tiene : 1 Dolor de cabeza migrañoso: dolor punzante intenso en un solo lado de la cabeza. 2 Dolores de cabeza en racimo - dolores de cabeza severos y recurrentes 3 Dolor de cabeza por tensión - Ocasional
humano: rasimo
Bot:  síntoma observado. Los ataques tienden a ocurrir en grupos diarios que pueden persistir durante semanas o meses. Por lo general, tienen lugar a la misma hora del día, que a menudo puede ser un par de horas después de quedarse dormido por la noche
humano: asios
Bot:  no tengo certerza de ello
humano: adios
Bot:  fue un gusto
